In [29]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded successfully!")


Model loaded successfully!


In [30]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import os

In [31]:
# 3. Load your multilingual E5 model
model = SentenceTransformer("intfloat/multilingual-e5-base")
print("✅ Model loaded.")

✅ Model loaded.


In [32]:
# 4. Load your dataset
data_path = r"C:\Users\ASUS-PC\Desktop\mindcheck_ai_jupyter\data\aimind_cleaned.csv"
df = pd.read_csv(data_path)

C:\Users\ASUS-PC\AppData\Local\Temp\ipykernel_16924\2523802732.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [33]:
# Ensure there’s a 'text' column
assert "confession" in df.columns, "❌ 'confession' column not found in dataset"
df = df.dropna(subset=["confession"]).reset_index(drop=True)

In [34]:
print("✅ Data loaded. Sample:")
print(df[["confession"]].head())

✅ Data loaded. Sample:
                                          confession
0  New to Vyvanse... Do others get a heady rush f...
1  why FUCK is a mental illness supposed to stay ...
2  PTSD? Hello. Sorry for my bad english but I ju...
3  Found mould in my home Hi all, contamination O...
4                             Honestly I wanna die. 


In [ ]:
# 05_faiss_index_building.ipynb

import numpy as np
import faiss
import pickle

# === 1. Load precomputed embeddings and texts ===
embeddings = np.load("../data/embeddings.npy")  # Path to embeddings from Notebook 03
with open("../data/texts.pkl", "rb") as f:
    texts = pickle.load(f)

print(f"Loaded {len(texts)} texts with shape {embeddings.shape}")

# === 2. Normalize embeddings for cosine similarity ===
embeddings = embeddings.astype("float32")
faiss.normalize_L2(embeddings)

# === 3. Create FAISS index ===
index = faiss.IndexFlatIP(embeddings.shape[1])  # Inner Product = Cosine after normalization
index.add(embeddings)

print(f"FAISS index contains {index.ntotal} vectors.")

# === 4. Save index and texts ===
faiss.write_index(index, "../data/mental_health.index")
with open("../data/texts.pkl", "wb") as f:
    pickle.dump(texts, f)

print("✅ FAISS index saved to ../data/mental_health.index")


In [35]:
texts = df["confession"].astype(str).apply(lambda x: f"passage: {x}").tolist()

In [36]:
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    batch_size=32,
    normalize_embeddings=True
)

Batches: 100%|██████████| 4657/4657 [5:05:34<00:00,  3.94s/it]  


In [37]:
# 7. Save embeddings
output_embeddings = np.array(embeddings)
output_ids = df.get("id", df.index)  # use 'id' column if exists, else use index


In [38]:
# 8. Create output DataFrame
emb_df = pd.DataFrame(output_embeddings)
emb_df["id"] = output_ids

In [13]:
# 9. Save to CSV
output_path = r"C:\Users\ASUS-PC\Desktop\mindcheck_ai_jupyter\data\aimind_embeddings.csv"
emb_df.to_csv(output_path, index=False)

print(f"✅ Embeddings saved to: {output_path}")

✅ Embeddings saved to: C:\Users\ASUS-PC\Desktop\mindcheck_ai_jupyter\data\aimind_embeddings.csv
